In [ ]:
# Base Worker for Megadrive transfer

from google.colab import output
import logging


class Megadrive(BaseWorker):
    def __init__(self, name, state):
        super().__init__(name, state)
        self.files = ['list.txt']
        self.file_completed = 0
        self.file_onqueue = []
        self.file_total = 0
        self.file_working = 0
        self.file_current = ''

    def working(self):
        # Initiation
        print('----- Initiation -----')
        logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] - %(message)s')
        if not self.init():
            self.success = False
            return
        output.clear()
        print('Initiation Done ...')
        # Read files
        print('----- Reading files -----')
        with open('/content/list.txt', 'r') as f:
            rl = f.read().split('\n')
            raw_list = [raw for raw in rl if raw != '']
        # Check files
        print('----- Checking files -----')
        for raw in raw_list:
            if self.check(raw):
                self.file_completed += 1
            else:
                self.file_onqueue.append(raw)
        self.file_total = len(self.file_onqueue)
        print(f'Checking done. {self.file_completed} files completed. {self.file_total} files on queue.')
        if self.file_total > 0:
            self.log('INFO', f'{self.file_completed} files completed. {self.file_total} files on queue.')
            self.endmsg = f'Completed. {self.file_total} transfered. {self.file_completed} files completed'
        else:
            self.endmsg = f'Nothing to transfer. {self.file_completed} files completed.'
            return
        # Transfer files
        print('----- Transferring files -----')
        for file_path in self.file_onqueue:
            self.file_working += 1
            self.file_current = file_path
            self.smalllog()
            self.transfer(file_path)
        # Done
        print('----- DONE -----')

    def get_result_path(self, raw_path, from_path, to_path):
        parent_path, file_name = os.path.split(raw_path)
        if parent_path.startswith(from_path):
            parent_path = parent_path[len(from_path):]
        result_folder = os.path.join(to_path, parent_path.lstrip('/'))
        result_file = os.path.join(result_folder, file_name)
        return result_folder, result_file

    
    def init(self):
        # Implement this. Return True when initiation is completed, return False otherwise
        pass
    
    def check(self, path):
        # Implement this. Return True when the file is already existed, return False otherwise
        pass
    
    def transfer(self, path):
        # Implement this
        pass

    def smalllog(self):
        # Implement this
        msg = 'Transfering: ' + str(self.file_working) + ' / ' + str(self.file_total) + '\nCompleted: ' + str(self.file_completed) + '\n' + self.file_current
        print()
        logging.info(msg)
